<a href="https://colab.research.google.com/github/candido05/ML_Projects/blob/main/english_portuguese_translate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
nageshsingh_englishportuguese_translation_path = kagglehub.dataset_download('nageshsingh/englishportuguese-translation')

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/englishportuguese-translation/por.txt


In [ ]:
import warnings
import os
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import re
import unicodedata

In [ ]:
import nltk
nltk.download("all")

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /usr/share/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /usr/share/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /usr/share/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     /usr/share/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_eng is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /usr/share/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package ave

True

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
filepath = "/kaggle/input/englishportuguese-translation/por.txt"

In [ ]:
df = pd.read_csv(filepath, sep="\t", names=["En", "Pt", "NAN"])[["En", "Pt"]]
df.head(10)

,En,Pt
0,Go.,Vai.
1,Go.,Vá.
2,Hi.,Oi.
3,Run!,Corre!
4,Run!,Corra!
5,Run!,Corram!
6,Run.,Corre!
7,Run.,Corra!
8,Run.,Corram!
9,Who?,Quem?


In [ ]:
df.shape

(168903, 2)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 168903 entries, 0 to 168902
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   En      168903 non-null  object
 1   Pt      168903 non-null  object
dtypes: object(2)
memory usage: 2.6+ MB


In [ ]:
df.duplicated().sum()

0

In [ ]:
df = df.sample(frac=1).reset_index(drop=True)
df.head(10)

,En,Pt
0,Tom wasn't the only one crying.,Tom não era o único que chorava.
1,The farmer employed five new workers.,O fazendeiro contratou cinco novos funcionários.
2,I can't believe that you really believe that.,Eu não acredito que você realmente acredita ni...
3,I come in peace.,Venho em paz.
4,Many thanks for your help.,Muito obrigado pela sua ajuda.
5,What's that in your hand?,Que é isso em sua mão?
6,What a nice guy!,Que cara bacana!
7,Tom knows who won.,Tom sabe quem ganhou.
8,I'm curious.,Quero saber.
9,Stay here until I get back.,Fique aqui até eu voltar.


In [ ]:
data = df[:100000].copy()

# Modificar a coluna de forma segura usando .loc
data.loc[:, 'Pt'] = data['Pt'].apply(lambda x: "<SOS> " + x + " <EOS>")
data.head()

,En,Pt
0,Tom wasn't the only one crying.,<SOS> Tom não era o único que chorava. <EOS>
1,The farmer employed five new workers.,<SOS> O fazendeiro contratou cinco novos funci...
2,I can't believe that you really believe that.,<SOS> Eu não acredito que você realmente acred...
3,I come in peace.,<SOS> Venho em paz. <EOS>
4,Many thanks for your help.,<SOS> Muito obrigado pela sua ajuda. <EOS>


In [ ]:
def eng_preprocess(text):
    text = text.lower()
    text = re.sub(r'[^a-z0-9\s]', '', text)  # Corrigido regex e typo
    text = nltk.word_tokenize(text)
    text = " ".join([i.strip() for i in text])
    return text

def pt_preprocess(text):
    text = text.replace("\u202f", " ")
    text = text.lower()
    text = re.sub(r'[^a-zéâàçêëôîû!?,\'\s]', ' ', text)  # Ajustado regex
    return text

In [ ]:
data['En'] = data['En'].apply(lambda x: eng_preprocess(x))

In [ ]:
data.head()

,En,Pt
0,tom wasnt the only one crying,<SOS> Tom não era o único que chorava. <EOS>
1,the farmer employed five new workers,<SOS> O fazendeiro contratou cinco novos funci...
2,i cant believe that you really believe that,<SOS> Eu não acredito que você realmente acred...
3,i come in peace,<SOS> Venho em paz. <EOS>
4,many thanks for your help,<SOS> Muito obrigado pela sua ajuda. <EOS>


In [ ]:
data['Pt'] = data['Pt'].apply(lambda x: pt_preprocess(x))

In [ ]:
data.head()

,En,Pt
0,tom wasnt the only one crying,sos tom n o era o nico que chorava eos
1,the farmer employed five new workers,sos o fazendeiro contratou cinco novos funci...
2,i cant believe that you really believe that,sos eu n o acredito que você realmente acred...
3,i come in peace,sos venho em paz eos
4,many thanks for your help,sos muito obrigado pela sua ajuda eos


In [ ]:
def feature_tokenization(feat):
    eng_token = Tokenizer()
    eng_token.fit_on_texts(feat)
    eng_seq = eng_token.texts_to_sequences(feat)
    return eng_seq, eng_token

def target_tokenization(target):
    port_token = Tokenizer()
    port_token.fit_on_texts(target)
    port_seq = port_token.texts_to_sequences(target)
    return port_seq, port_token

In [ ]:
eng_sequences, eng_token = feature_tokenization(data['En'])

In [ ]:
eng_vocab = len(eng_token.word_index) + 1
eng_vocab

10886

In [ ]:
eng_sequences[:5]

[[1, 121, 5, 146, 60, 643],
 [5, 2051, 5578, 524, 155, 2134],
 [2, 63, 221, 8, 4, 103, 221, 8],
 [2, 92, 10, 1536],
 [128, 527, 25, 30, 69]]

In [ ]:
port_sequences, port_token = target_tokenization(data['Pt'])

In [ ]:
port_vocab = len(port_token.word_index) + 1
port_vocab

16444

In [ ]:
port_sequences[:5]

[[1, 4, 5, 3, 82, 3, 424, 6, 3757, 2],
 [1, 3, 3064, 2444, 470, 1518, 4068, 480, 2],
 [1, 7, 5, 3, 516, 6, 10, 158, 895, 473, 2],
 [1, 1724, 21, 1192, 2],
 [1, 25, 362, 237, 54, 201, 2]]

In [ ]:
port_inp = [x[:-1] for x in port_sequences]
port_inp[:5]

[[1, 4, 5, 3, 82, 3, 424, 6, 3757],
 [1, 3, 3064, 2444, 470, 1518, 4068, 480],
 [1, 7, 5, 3, 516, 6, 10, 158, 895, 473],
 [1, 1724, 21, 1192],
 [1, 25, 362, 237, 54, 201]]

In [ ]:
port_out = [x[1:] for x in port_sequences]
port_out[:5]

[[4, 5, 3, 82, 3, 424, 6, 3757, 2],
 [3, 3064, 2444, 470, 1518, 4068, 480, 2],
 [7, 5, 3, 516, 6, 10, 158, 895, 473, 2],
 [1724, 21, 1192, 2],
 [25, 362, 237, 54, 201, 2]]

In [ ]:
def pad_seq(pad):
    return pad_sequences(pad, padding = "post")

In [ ]:
encoder_input = pad_seq(eng_sequences)
encoder_input = tf.convert_to_tensor(encoder_input)
encoder_input

<tf.Tensor: shape=(100000, 35), dtype=int32, numpy=
array([[    1,   121,     5, ...,     0,     0,     0],
       [    5,  2051,  5578, ...,     0,     0,     0],
       [    2,    63,   221, ...,     0,     0,     0],
       ...,
       [  462, 10885,  5161, ...,     0,     0,     0],
       [    1,    73,   111, ...,     0,     0,     0],
       [   32,   146,    38, ...,     0,     0,     0]], dtype=int32)>

In [ ]:
encoder_input.shape

TensorShape([100000, 35])

In [ ]:
decoder_input = pad_seq(port_inp)
decoder_input = tf.convert_to_tensor(decoder_input)
decoder_input

<tf.Tensor: shape=(100000, 35), dtype=int32, numpy=
array([[    1,     4,     5, ...,     0,     0,     0],
       [    1,     3,  3064, ...,     0,     0,     0],
       [    1,     7,     5, ...,     0,     0,     0],
       ...,
       [    1,   522, 16443, ...,     0,     0,     0],
       [    1,     4,   121, ...,     0,     0,     0],
       [    1,    20,     5, ...,     0,     0,     0]], dtype=int32)>

In [ ]:
decoder_input.shape

TensorShape([100000, 35])

In [ ]:
decoder_output = pad_seq(port_out)
decoder_output = tf.convert_to_tensor(decoder_output)
decoder_output

<tf.Tensor: shape=(100000, 35), dtype=int32, numpy=
array([[    4,     5,     3, ...,     0,     0,     0],
       [    3,  3064,  2444, ...,     0,     0,     0],
       [    7,     5,     3, ...,     0,     0,     0],
       ...,
       [  522, 16443,    36, ...,     0,     0,     0],
       [    4,   121,   137, ...,     0,     0,     0],
       [   20,     5,     3, ...,     0,     0,     0]], dtype=int32)>

In [ ]:
decoder_output.shape

TensorShape([100000, 35])

In [ ]:
eng_vocab = eng_vocab
port_vocab = port_vocab

eng_seq_len = encoder_input.shape[1]
port_seq_len = decoder_input.shape[1]

embed_dim = 180
hidden_dim = 128

num_heads = 6
head_dim = embed_dim//num_heads

num_blocks = 4

In [ ]:
class positional_embedding(tf.keras.layers.Layer):

    def __init__(self, vocab, sequence, embedding_dim, **kwargs):
        super().__init__(**kwargs)

        self.vocab = vocab
        self.sequence = sequence
        self.embedding_dim = embedding_dim

        self.embedding_layer = tf.keras.layers.Embedding(input_dim=self.vocab, output_dim=self.embedding_dim, input_length=self.sequence)
        self.positional_encoding = tf.keras.layers.Embedding(self.sequence, self.embedding_dim)


    def call(self, inputs):
        pos_ids = tf.range(self.sequence)
        token_embed = self.embedding_layer(inputs)
        token_positional = self.positional_encoding(pos_ids)
        final = token_embed + token_positional
        return  final

In [ ]:
class padding_mask(tf.keras.layers.Layer):

    def __init__(self, **kwargs):
        super().__init__(**kwargs)

    def call(self, inputs):
        mask = tf.cast(tf.math.not_equal(inputs, 0), tf.float32)
        mask = mask[:, tf.newaxis, :]
        return mask

In [ ]:
class look_ahead_mask(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.mask = padding_mask()

    def call(self, inputs, sequence_length):
        # Gera a máscara de padding
        masked = self.mask(inputs)  # Forma: [batch_size, 1, sequence_length]

        # Gera a máscara lookahead triangular
        look = tf.linalg.band_part(tf.ones((sequence_length, sequence_length)), -1, 0)  # Forma: [sequence_length, sequence_length]
        look = tf.expand_dims(tf.expand_dims(look, 0), 0)  # Forma: [1, 1, sequence_length, sequence_length]

        # Combina as máscaras (padding e lookahead)
        dec_mask = tf.minimum(masked[:, :, tf.newaxis, :], look)  # Forma: [batch_size, 1, sequence_length, sequence_length]

        return dec_mask

In [ ]:
class encoder_layer(tf.keras.layers.Layer):
    def __init__(self, num_heads, embedding_dim, hidden_dim, dropout_rate=0.0, **kwargs):
        super().__init__(**kwargs)

        self.mha = tf.keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=embedding_dim, dropout=dropout_rate)
        self.norm1 = tf.keras.layers.LayerNormalization()
        self.norm2 = tf.keras.layers.LayerNormalization()
        self.add = tf.keras.layers.Add()
        self.drop1 = tf.keras.layers.Dropout(dropout_rate)
        self.drop2 = tf.keras.layers.Dropout(dropout_rate)

        self.ffn = tf.keras.Sequential([
            tf.keras.layers.Dense(hidden_dim, activation="relu"),
            tf.keras.layers.Dense(hidden_dim, activation="relu"),
            tf.keras.layers.Dense(embedding_dim, activation="relu"),
            tf.keras.layers.Dropout(dropout_rate)
        ])

    def call(self, enc_inputs, enc_mask, training=False):  # 'training' como argumento nomeado
        mha_out1 = self.mha(enc_inputs, enc_inputs, enc_inputs, enc_mask)
        drop_out1 = self.drop1(mha_out1, training=training)
        norm_out1 = self.norm1(self.add([drop_out1, enc_inputs]))

        ffn_out = self.ffn(norm_out1)
        drop_out2 = self.drop2(ffn_out, training=training)
        final = self.norm2(self.add([drop_out2, norm_out1]))

        return final

In [ ]:
class Encoder(tf.keras.layers.Layer):
    def __init__(self, vocab, sequence, embedding_dim, num_heads, hidden_dim, num_blocks, dropout_rate=0.0, **kwargs):
        super().__init__(**kwargs)

        self.pos = positional_embedding(vocab, sequence, embedding_dim)
        self.mask = padding_mask()
        self.encoder = [encoder_layer(num_heads, embedding_dim, hidden_dim) for _ in range(num_blocks)]
        self.add = tf.keras.layers.Add()

    def call(self, inputs, training=False):  # 'training' como argumento nomeado
        emb = self.pos(inputs)
        mask = self.mask(inputs)
        temp = emb

        for blocks in self.encoder:
            emb = blocks(emb, mask, training=training)  # Passa 'training' como keyword argument
            emb = self.add([temp, emb])
            temp = emb

        return emb, mask

In [ ]:
class decoder_layer(tf.keras.layers.Layer):
    def __init__(self, num_heads, embedding_dim, hidden_dim, dropout_rate=0.0, **kwargs):
        super().__init__(**kwargs)

        self.mha1 = tf.keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=embedding_dim, dropout=dropout_rate)
        self.mha2 = tf.keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=embedding_dim, dropout=dropout_rate)
        self.norm1 = tf.keras.layers.LayerNormalization()
        self.norm2 = tf.keras.layers.LayerNormalization()
        self.norm3 = tf.keras.layers.LayerNormalization()
        self.add = tf.keras.layers.Add()
        self.drop1 = tf.keras.layers.Dropout(dropout_rate)
        self.drop2 = tf.keras.layers.Dropout(dropout_rate)
        self.drop3 = tf.keras.layers.Dropout(dropout_rate)

        self.ffn = tf.keras.Sequential([
            tf.keras.layers.Dense(hidden_dim, activation="relu"),
            tf.keras.layers.Dense(hidden_dim, activation="relu"),
            tf.keras.layers.Dense(embedding_dim, activation="relu"),
            tf.keras.layers.Dropout(dropout_rate)
        ])

    def call(self, dec_inputs, enc_outputs, dec_mask, enc_mask, training=False):
        mha_out1 = self.mha1(dec_inputs, dec_inputs, dec_inputs, attention_mask=dec_mask)
        drop_out1 = self.drop1(mha_out1, training=training)
        norm_out1 = self.norm1(self.add([drop_out1, dec_inputs]))

        mha_out2 = self.mha2(norm_out1, enc_outputs, enc_outputs, attention_mask=enc_mask)
        drop_out2 = self.drop2(mha_out2, training=training)
        norm_out2 = self.norm2(self.add([drop_out2, norm_out1]))

        ffn_out = self.ffn(norm_out2)
        drop_out3 = self.drop3(ffn_out, training=training)
        final = self.norm3(self.add([drop_out3, norm_out2]))

        return final

In [ ]:
class Decoder(tf.keras.layers.Layer):
    def __init__(self, vocab, sequence, embedding_dim, num_heads, hidden_dim, num_blocks, dropout_rate=0.1, **kwargs):
        super().__init__(**kwargs)

        self.sequence_length = sequence
        self.pos = positional_embedding(vocab, sequence, embedding_dim)
        self.look = look_ahead_mask()
        self.decoder = [decoder_layer(num_heads, embedding_dim, hidden_dim) for _ in range(num_blocks)]
        self.add = tf.keras.layers.Add()

    def call(self, dec_inputs, encoder_outputs, enc_mask, training=False):
        emb = self.pos(dec_inputs)
        mask = self.look(dec_inputs, sequence_length=self.sequence_length)
        temp = emb

        for blocks in self.decoder:
            emb = blocks(emb, encoder_outputs, mask, enc_mask, training=training)
            emb = self.add([temp, emb])
            temp = emb

        return emb

In [ ]:
class Transformer(tf.keras.Model):
    def __init__(self, eng_vocab, port_vocab, eng_seq_len, port_seq_len, embedding_dim, hidden_dim, num_heads, num_blocks, dropout_rate=0.1, **kwargs):
        super().__init__(**kwargs)

        self.encoder = Encoder(eng_vocab, eng_seq_len, embedding_dim, num_heads, hidden_dim, num_blocks)
        self.decoder = Decoder(port_vocab, port_seq_len, embedding_dim, num_heads, hidden_dim, num_blocks)

        self.Final_layer = tf.keras.layers.Dense(port_vocab, activation='relu')
        self.softmax = tf.keras.layers.Softmax(axis=-1)

    def call(self, inputs, training=False):
        enc_inputs, dec_inputs = inputs

        encoder_out, enc_mask = self.encoder(enc_inputs, training=training)
        decoder_out = self.decoder(dec_inputs, encoder_out, enc_mask, training=training)

        final = self.Final_layer(decoder_out)
        softmaxed = self.softmax(final)

        return softmaxed

In [ ]:
transformers = Transformer(eng_vocab, port_vocab, eng_seq_len,
                           port_seq_len, embed_dim, hidden_dim,
                           num_heads, num_blocks)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [ ]:
trans_out = transformers((encoder_input[:1], decoder_input[:1]), training=False)

In [ ]:
trans_out

<tf.Tensor: shape=(1, 35, 16444), dtype=float32, numpy=
array([[[4.51336127e-05, 4.51336127e-05, 1.04714789e-04, ...,
         4.51336127e-05, 4.51336127e-05, 4.51336127e-05],
        [4.79636874e-05, 4.47269449e-05, 4.93040425e-05, ...,
         9.50788672e-05, 4.47269449e-05, 6.78518118e-05],
        [4.52583881e-05, 4.86424578e-05, 4.52583881e-05, ...,
         4.96925168e-05, 4.52583881e-05, 4.52583881e-05],
        ...,
        [7.68870377e-05, 9.97956231e-05, 5.71735181e-05, ...,
         4.50536427e-05, 4.50536427e-05, 4.50536427e-05],
        [4.53077446e-05, 1.14522845e-04, 4.76471760e-05, ...,
         4.53077446e-05, 4.53077446e-05, 5.49684773e-05],
        [5.38023633e-05, 1.50434207e-04, 4.52935965e-05, ...,
         4.52935965e-05, 4.52935965e-05, 4.83688091e-05]]], dtype=float32)>

In [ ]:
transformers.summary()

Model: "transformer"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ encoder (Encoder)                    │ ?                           │       5,344,340 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ decoder (Decoder)                    │ ?                           │       9,470,300 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_24 (Dense)                     │ (1, 35, 16444)              │       2,976,364 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ softmax (Softmax)                    │ ?                           │               0 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 17,791,004 (67.87 MB)

 Trainable params: 17,791,004 (67.87 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
transformers.get_build_config()

{'input_shape': ((1, 35), (1, 35))}

In [ ]:
path = "/kaggle/working/model.weights.h5"
transformers.save_weights(path)

In [ ]:
compilation = transformers.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(),optimizer=tf.keras.optimizers.RMSprop() ,metrics=["accuracy"])

In [ ]:
calls = tf.keras.callbacks.EarlyStopping(patience = 5, verbose = True)

In [ ]:
fitting = transformers.fit((encoder_input,decoder_input),decoder_output, validation_split=0.2, epochs = 100, batch_size = 500, verbose = True, callbacks = calls)

Epoch 1/100
  7/160 ━━━━━━━━━━━━━━━━━━━━ 2:13:50 52s/step - accuracy: 0.3022 - loss: 7.7210